In [1]:
import tech_indicators as ti
import pandas as pd
import numpy as np
import datetime as dt
from statsmodels.tsa.seasonal import STL

stock_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

df_spy_daily = pd.read_csv("../stock_price/Daily/SPY.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'SPY'})
df_spy_daily['Date'] = pd.to_datetime(df_spy_daily['Date'])
df_spy_monthly = pd.read_csv("../stock_price/Monthly/SPY.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'SPY'})
df_spy_monthly['Date'] = pd.to_datetime(df_spy_monthly['Date'])
df_spy_weekly = pd.read_csv("../stock_price/Weekly/SPY.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'SPY'})
df_spy_weekly['Date'] = pd.to_datetime(df_spy_weekly['Date'])

df_ndaq_daily = pd.read_csv("../stock_price/Daily/NDAQ.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'NDAQ'})
df_ndaq_daily['Date'] = pd.to_datetime(df_ndaq_daily['Date'])
df_ndaq_monthly = pd.read_csv("../stock_price/Monthly/NDAQ.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'NDAQ'})
df_ndaq_monthly['Date'] = pd.to_datetime(df_ndaq_monthly['Date'])
df_ndaq_weekly = pd.read_csv("../stock_price/Weekly/NDAQ.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'NDAQ'})
df_ndaq_weekly['Date'] = pd.to_datetime(df_ndaq_weekly['Date'])

df_ndaq_daily 

,Date,NDAQ
0,2007-01-03,8.575854
1,2007-01-04,8.819220
2,2007-01-05,9.153847
3,2007-01-08,9.413805
4,2007-01-09,9.355729
...,...,...
3288,2020-01-27,34.929343
3289,2020-01-28,35.362062
3290,2020-01-29,36.698400
3291,2020-01-30,36.895669


In [2]:
stock = 'AAPL'
stock_df = pd.read_csv("../stock_price/Daily/"+stock+".csv")
df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
df.reset_index(inplace=True)

macd = ti.calculate_vwap(df, period = 14).iloc[:,0]
macd.head(30)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13    90.983805
14    90.808569
15    90.661339
16    90.472963
17    90.317893
18    90.218006
19    90.065164
20    89.950401
21    89.832974
22    89.724210
23    89.553937
24    89.409635
25    89.337278
26    89.202146
27    89.101188
28    89.026735
29    88.969714
Name: vwap, dtype: float64

In [3]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Daily/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df.reset_index(inplace=True)
    sma = ti.calculate_sma(df, window=14)
    ema = ti.calculate_ema(df, window=14)
    k, d = ti.calculate_stoch(df)
    rsi = ti.calculate_rsi(df)
    mfi = ti.calculate_mfi(df)
    sar = ti.calculate_sar(df)
    ad = ti.calc_AD(df)
    macd = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,0]
    macd_signal = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,1]
    macd_histo = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,2]
    vmap = ti.calculate_vwap(df, period = 14).iloc[:,0]

    tech_Daily = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad,
                "MACD":macd,
                "MACD_Signal":macd_signal,
                "MACD_Histo":macd_histo,
                "VWAP":vmap
              })
    tech_Daily['Date'] = pd.to_datetime(tech_Daily['Date'])
    tech_Daily = tech_Daily.loc[(tech_Daily['Date']>='2010-01-01') & (tech_Daily['Date']<='2020-02-01')]
    tech_Daily = tech_Daily.reset_index(drop=True)
    tech_Daily = pd.merge(tech_Daily,df_spy_daily,on='Date',how='left')
    tech_Daily = pd.merge(tech_Daily,df_ndaq_daily,on='Date',how='left')
    tech_Daily.to_csv("Tech/Daily/"+stock+".csv",index=0)
tech_Daily 

,Date,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,MACD,MACD_Signal,MACD_Histo,VWAP,SPY,NDAQ
0,2010-01-04,87.050714,86.469171,8800.533871,8780.517509,74.637681,79.040088,2.029269e+04,1.290264e+09,1.994113,1.997987,-0.003874,67.021223,88.050132,5.597397
1,2010-01-05,87.219286,86.557282,8698.509499,8745.184233,60.369069,73.772836,2.069679e+04,1.218850e+09,1.875632,1.973516,-0.097884,67.048539,88.283213,5.613990
2,2010-01-06,87.206429,86.477644,8575.594272,8691.545881,49.194991,70.378941,2.110901e+04,1.151087e+09,1.668097,1.912432,-0.244336,67.065191,88.345367,5.611225
3,2010-01-07,87.187143,86.515292,8655.594272,8643.232681,48.782687,70.096512,2.152947e+04,1.223507e+09,1.550306,1.840007,-0.289701,67.093830,88.718297,5.536556
4,2010-01-08,87.185000,86.579919,8679.335749,8636.841431,49.867608,75.080973,2.195835e+04,1.296902e+09,1.459498,1.763905,-0.304407,67.106328,89.013532,5.594632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532,2020-01-27,199.490000,200.234024,20160.240130,20431.981364,74.112654,80.031356,1.205512e+26,5.225094e+10,5.202679,4.650774,0.551906,104.020698,307.043805,34.929343
2533,2020-01-28,200.501429,200.582821,20276.148617,20309.413576,76.636569,80.540753,1.229622e+26,5.285468e+10,4.955482,4.711715,0.243766,104.055439,310.261359,35.362062
2534,2020-01-29,201.425714,201.153111,20475.067575,20303.818774,75.512618,80.810211,1.254215e+26,5.346109e+10,4.865677,4.742508,0.123170,104.089879,310.005094,36.698400
2535,2020-01-30,202.494286,202.094030,20809.653937,20520.290043,77.319211,81.152525,1.279299e+26,5.407506e+10,5.007105,4.795427,0.211678,104.132440,311.011172,36.895669


In [4]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Weekly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df.reset_index(inplace=True)
    sma = ti.calculate_sma(df, window=14)
    ema = ti.calculate_ema(df, window=14)
    k, d = ti.calculate_stoch(df)
    rsi = ti.calculate_rsi(df)
    mfi = ti.calculate_mfi(df)
    sar = ti.calculate_sar(df)
    ad = ti.calc_AD(df)
    macd = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,0]
    macd_signal = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,1]
    macd_histo = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,2]
    vmap = ti.calculate_vwap(df, period = 14).iloc[:,0]

    tech_Weekly = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad,
                "MACD":macd,
                "MACD_Signal":macd_signal,
                "MACD_Histo":macd_histo,
                "VWAP":vmap
              })
    tech_Weekly['Date'] = pd.to_datetime(tech_Weekly['Date'])
    tech_Weekly = tech_Weekly.loc[(tech_Weekly['Date']>='2010-01-01') & (tech_Weekly['Date']<='2020-02-01')]
    tech_Weekly = tech_Weekly.reset_index(drop=True)
    tech_Weekly = pd.merge(tech_Weekly,df_spy_weekly,on='Date',how='left')
    tech_Weekly = pd.merge(tech_Weekly,df_ndaq_weekly,on='Date',how='left')
    tech_Weekly.to_csv("Tech/Weekly/"+stock+".csv",index=0)
tech_Weekly 

,Date,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,MACD,MACD_Signal,MACD_Histo,VWAP,SPY,NDAQ
0,2010-01-08,80.602857,81.179788,8696.976223,8688.741962,87.013487,82.878940,60.720900,3.861826e+09,5.522472,4.781590,0.740882,67.373099,89.0135,5.5946
1,2010-01-15,81.545714,81.841149,8609.794544,8683.298533,80.470914,77.778757,61.281882,3.371076e+09,5.433972,4.912066,0.521905,67.457450,88.2910,5.5753
2,2010-01-22,82.148571,81.959663,8268.794544,8525.188437,67.790894,72.713330,61.815045,2.894890e+09,5.030685,4.935790,0.094895,67.537214,84.8492,5.0720
3,2010-01-29,82.713571,81.969041,8198.794544,8359.127878,66.309278,68.044877,62.346944,2.425818e+09,4.601550,4.868942,-0.267392,67.604273,83.4351,4.9752
4,2010-02-05,83.199286,82.047836,8250.542837,8239.377308,64.693172,67.422754,62.799605,2.882981e+09,4.255172,4.746188,-0.491016,67.746003,82.8680,5.1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,2020-01-03,181.753571,183.104288,18952.523725,18891.666128,78.363636,67.088343,155.217186,1.200533e+11,4.965629,4.605022,0.360608,103.906016,306.0093,34.4521
522,2020-01-10,183.024286,184.526383,19370.022550,19084.543751,79.959582,76.184441,155.935642,1.227405e+11,5.390533,4.762124,0.628409,104.052214,309.1414,33.9939
523,2020-01-17,184.998571,187.216199,20465.077235,19595.874503,84.951947,77.424269,156.708329,1.254778e+11,6.533913,5.116482,1.417431,104.294903,315.0640,34.7798
524,2020-01-24,187.090714,189.587372,20495.740175,20110.279987,88.048844,78.123002,157.669162,1.281907e+11,7.379194,5.569024,1.810170,104.492434,312.0457,35.0662


In [5]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Monthly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df.reset_index(inplace=True)
    sma = ti.calculate_sma(df, window=14)
    ema = ti.calculate_ema(df, window=14)
    k, d = ti.calculate_stoch(df)
    rsi = ti.calculate_rsi(df)
    mfi = ti.calculate_mfi(df)
    sar = ti.calculate_sar(df)
    ad = ti.calc_AD(df)
    macd = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,0]
    macd_signal = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,1]
    macd_histo = ti.calculate_macd(df, ema_short=12, ema_long=26, signal_period=9).iloc[:,2]
    vmap = ti.calculate_vwap(df, period = 14).iloc[:,0]

    tech_Monthly = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad,
                "MACD":macd,
                "MACD_Signal":macd_signal,
                "MACD_Histo":macd_histo,
                "VWAP":vmap
              })
    tech_Monthly['Date'] = pd.to_datetime(tech_Monthly['Date'])
    tech_Monthly = tech_Monthly.loc[(tech_Monthly['Date']>='2010-01-01') & (tech_Monthly['Date']<='2020-02-01')]
    tech_Monthly = tech_Monthly.reset_index(drop=True)
    tech_Monthly = pd.merge(tech_Monthly,df_spy_monthly,on='Date',how='left')
    tech_Monthly = pd.merge(tech_Monthly,df_ndaq_monthly,on='Date',how='left')
    tech_Monthly.to_csv("Tech/Monthly/"+stock+".csv",index=0)
tech_Monthly 

,Date,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,MACD,MACD_Signal,MACD_Histo,VWAP,SPY,NDAQ
0,2010-01-29,67.211429,72.957338,8202.127948,8348.738778,73.077525,65.332522,46.421880,2.338943e+09,0.973976,-2.347974,3.321950,68.161205,83.4351,4.9752
1,2010-02-26,69.556429,74.600360,8527.127948,8491.461282,74.503657,65.571182,47.268843,3.551670e+09,1.832523,-1.511875,3.344398,68.518909,86.0379,5.1521
2,2010-03-31,72.533571,76.790978,9101.968540,8610.408145,79.925330,71.567999,48.078666,4.808868e+09,2.942979,-0.620904,3.563883,69.060474,91.2780,5.8408
3,2010-04-30,74.927857,78.582848,9021.994056,8883.696848,76.569436,71.092458,48.989693,3.629440e+09,3.715639,0.246404,3.469234,69.467429,92.6901,5.8076
4,2010-05-28,76.132143,77.766468,7244.768881,8456.243826,60.571859,65.259243,49.953739,2.660060e+09,2.861106,0.769345,2.091761,69.978007,85.3254,5.1411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2019-09-30,155.567143,157.169823,17199.142093,17692.416342,67.103094,60.260111,99.085267,8.965963e+10,16.460580,13.721370,2.739209,102.413949,280.3014,31.4687
117,2019-10-31,157.850714,160.061847,17884.142093,17721.112551,66.015429,53.563570,100.844562,9.357406e+10,16.734561,14.324008,2.410553,103.020169,286.4974,31.6018
118,2019-11-29,160.309286,163.321601,18449.142093,17844.142093,66.829650,53.871889,102.431271,9.761312e+10,17.209224,14.901052,2.308172,103.549573,296.8681,33.1950
119,2019-12-31,163.884286,166.598720,18788.219359,18373.834515,76.790494,62.594108,104.079645,1.017502e+11,17.655421,15.451926,2.203496,104.208086,305.4872,34.0766


In [6]:
# PCA 
import numpy as np
import pandas as pd
import sys
from scipy.sparse import dia_matrix
import scipy.sparse.linalg
import math
from pca import pca
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from fancyimpute import KNN
from sklearn.impute import KNNImputer

features = list(tech_Daily.columns[1:])
features

['SMA',
 'EMA',
 'STOCH_k',
 'STOCK_d',
 'RSI',
 'MFI',
 'SAR',
 'AD',
 'MACD',
 'MACD_Signal',
 'MACD_Histo',
 'VWAP',
 'SPY',
 'NDAQ']

In [7]:
# small test 
def knnpca(df, k=5, n_components = 3):
    x = df.loc[:,features].values
    x = StandardScaler().fit_transform(x)
    x = KNN(k).fit_transform(x)
    pca = PCA(n_components)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
                , columns = ['PC{}'.format(i) for i in range(1, n_components+1)])
    
    return principalDf

knnpca(tech_Daily)

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/2537 with 0 missing, elapsed time: 0.550
Imputing row 101/2537 with 0 missing, elapsed time: 0.551
Imputing row 201/2537 with 0 missing, elapsed time: 0.551
Imputing row 301/2537 with 0 missing, elapsed time: 0.551
Imputing row 401/2537 with 0 missing, elapsed time: 0.551
Imputing row 501/2537 with 0 missing, elapsed time: 0.551
Imputing row 601/2537 with 0 missing, elapsed time: 0.551
Imputing row 701/2537 with 0 missing, elapsed time: 0.551
Imputing row 801/2537 with 0 missing, elapsed time: 0.552
Imputing row 901/2537 with 0 missing, elapsed time: 0.552
Imputing row 1001/2537 with 0 missing, elapsed time: 0.552
Imputing row 1101/2537 with 0 missing, elapsed time: 0.552
Imputing row 1201/2537 with 0 missing, elapsed time: 0.552
Imputing row 1301/2537 with 0 missing, elapsed time: 0.552
Imputing row 1401/2537 with 0 missing, elapsed time: 0.552
Imputing row 1501/2537 with 0 missing, elapsed time: 0.553
Imputing row 1601/2537 with 0 missing, elapsed time: 0.553
Imputing 

,PC1,PC2,PC3
0,-2.239673,-1.343091,-2.090098
1,-2.404530,-1.248360,-1.433816
2,-2.542289,-1.165266,-0.916386
3,-2.549310,-1.152541,-0.867331
4,-2.504489,-1.163611,-1.043698
...,...,...,...
2532,7.394892,2.895878,-1.982764
2533,7.505276,2.981866,-2.005029
2534,7.616858,3.086628,-1.955896
2535,7.761093,3.095505,-2.051154


In [8]:
# n = 0.99
n = 2

for stock in stock_list:
    stock_df = pd.read_csv("Tech/Daily/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df1 = df.loc[:,features]
    df1 = df1.dropna()
    df1 = df1.reset_index(drop=True)
    df_pca = knnpca(df1, n_components = n)
    df = pd.concat([df,df_pca],axis=1)
    df.to_csv("Tech/Daily_PCA/"+stock+".csv",index=0)

for stock in stock_list:
    stock_df = pd.read_csv("Tech/Weekly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df1 = df.loc[:,features]
    df1 = df1.dropna()
    df1 = df1.reset_index(drop=True)
    df_pca = knnpca(df1, n_components = n)
    df = pd.concat([df,df_pca],axis=1)
    df.to_csv("Tech/Weekly_PCA/"+stock+".csv",index=0)

for stock in stock_list:
    stock_df = pd.read_csv("Tech/Monthly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df1 = df.loc[:,features]
    df1 = df1.dropna()
    df1 = df1.reset_index(drop=True)
    df_pca = knnpca(df1, n_components = n)
    df = pd.concat([df,df_pca],axis=1)
    df.to_csv("Tech/Monthly_PCA/"+stock+".csv",index=0)

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/2537 with 0 missing, elapsed time: 0.548
Imputing row 101/2537 with 0 missing, elapsed time: 0.548
Imputing row 201/2537 with 0 missing, elapsed time: 0.548
Imputing row 301/2537 with 0 missing, elapsed time: 0.548
Imputing row 401/2537 with 0 missing, elapsed time: 0.549
Imputing row 501/2537 with 0 missing, elapsed time: 0.549
Imputing row 601/2537 with 0 missing, elapsed time: 0.549
Imputing row 701/2537 with 0 missing, elapsed time: 0.549
Imputing row 801/2537 with 0 missing, elapsed time: 0.549
Imputing row 901/2537 with 0 missing, elapsed time: 0.549
Imputing row 1001/2537 with 0 missing, elapsed time: 0.549
Imputing row 1101/2537 with 0 missing, elapsed time: 0.550
Imputing row 1201/2537 with 0 missing, elapsed time: 0.550
Imputing row 1301/2537 with 0 missing, elapsed time: 0.550
Imputing row 1401/2537 with 0 missing, elapsed time: 0.550
Imputing row 1501/2537 with 0 missing, elapsed time: 0.550
Imputing row 1601/2537 with 0 missing, elapsed time: 0.550
Imputing 

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/2537 with 0 missing, elapsed time: 0.558
Imputing row 101/2537 with 0 missing, elapsed time: 0.558
Imputing row 201/2537 with 0 missing, elapsed time: 0.559
Imputing row 301/2537 with 0 missing, elapsed time: 0.559
Imputing row 401/2537 with 0 missing, elapsed time: 0.559
Imputing row 501/2537 with 0 missing, elapsed time: 0.559
Imputing row 601/2537 with 0 missing, elapsed time: 0.559
Imputing row 701/2537 with 0 missing, elapsed time: 0.559
Imputing row 801/2537 with 0 missing, elapsed time: 0.559
Imputing row 901/2537 with 0 missing, elapsed time: 0.559
Imputing row 1001/2537 with 0 missing, elapsed time: 0.560
Imputing row 1101/2537 with 0 missing, elapsed time: 0.560
Imputing row 1201/2537 with 0 missing, elapsed time: 0.560
Imputing row 1301/2537 with 0 missing, elapsed time: 0.560
Imputing row 1401/2537 with 0 missing, elapsed time: 0.560
Imputing row 1501/2537 with 0 missing, elapsed time: 0.560
Imputing row 1601/2537 with 0 missing, elapsed time: 0.560
Imputing 

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


...

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/526 with 0 missing, elapsed time: 0.025
Imputing row 101/526 with 0 missing, elapsed time: 0.025
Imputing row 201/526 with 0 missing, elapsed time: 0.026
Imputing row 301/526 with 0 missing, elapsed time: 0.026
Imputing row 401/526 with 0 missing, elapsed time: 0.026
Imputing row 501/526 with 0 missing, elapsed time: 0.027
Imputing row 1/485 with 0 missing, elapsed time: 0.023
Imputing row 101/485 with 0 missing, elapsed time: 0.023
Imputing row 201/485 with 0 missing, elapsed time: 0.023
Imputing row 301/485 with 0 missing, elapsed time: 0.023
Imputing row 401/485 with 0 missing, elapsed time: 0.023
Imputing row 1/387 with 0 missing, elapsed time: 0.021
Imputing row 101/387 with 0 missing, elapsed time: 0.021
Imputing row 201/387 with 0 missing, elapsed time: 0.021
Imputing row 301/387 with 0 missing, elapsed time: 0.021
Imputing row 1/526 with 0 missing, elapsed time: 0.022
Imputing row 101/526 with 0 missing, elapsed time: 0.023
Imputing row 201/526 with 0 missing, el

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/so

Imputing row 1/526 with 0 missing, elapsed time: 0.024
Imputing row 101/526 with 0 missing, elapsed time: 0.025
Imputing row 201/526 with 0 missing, elapsed time: 0.025
Imputing row 301/526 with 0 missing, elapsed time: 0.025
Imputing row 401/526 with 0 missing, elapsed time: 0.025
Imputing row 501/526 with 0 missing, elapsed time: 0.026
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/55 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/100 with 0 missing, elapsed tim

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/so